In [9]:
import bqplot as bq

import pandas as pd

import ipywidgets as widgets
import numpy as np
from bqplot import pyplot as plt
import bqplot
from ipywidgets import HBox

from clustergrammer2 import net

In [10]:
scatter_size = 10

In [11]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [13]:
cat_colors = net.load_json_to_dict('../data/citeseq-dashboard/category_colors.json')

In [14]:
# read metadata
df_meta = pd.read_parquet('../data/citeseq-dashboard/meta_cell.parquet')

df_meta['merge-umap-x-pos'] = df_meta['merge-umap-x'] * 130 + 2500
df_meta['merge-umap-y-pos'] = df_meta['merge-umap-y'] * 130 + 2000

df_meta['adt-umap-x-pos'] = df_meta['adt-umap-x']     * 130 + 2000
df_meta['adt-umap-y-pos'] = df_meta['adt-umap-y']     * 130 + 1700

df_meta['gex-umap-x-pos'] = df_meta['gex-umap-x']     * 130 + 2700
df_meta['gex-umap-y-pos'] = df_meta['gex-umap-y']     * 130 + 2100

rows = sorted(df_meta.index.tolist())
df_meta = df_meta.loc[rows]

In [15]:
def scatter_observe(scatter, hover_data):
    scatter.colors = cell_type_colors
    
    inst_name = hover_data['data']['name']#.split('-')
    
    inst_cat = df_meta.loc[inst_name, 'Cell Type']
    
    list_opacities = []

    for inst_label in df_meta.index.tolist():
        inst_opacity = 0.15
        if df_meta.loc[inst_label, 'Cell Type'] == inst_cat:
            inst_opacity = 1
        list_opacities.append(inst_opacity)        

    # patch.opacities = list_opacities
    scatter.default_opacities = list_opacities
    
    fig.title = 'UMAP: ' + inst_cat

In [16]:
df_data = pd.read_parquet('../data/citeseq-dashboard/merge_data.parquet')
cols = sorted(df_data.columns.tolist())
df_data = df_data[cols]

In [17]:
# size_values = np.arcsinh(df_meta['UMI_Sum']/5)

In [18]:
# cat_colors['CD8_CD4_T_Cells'] = '#D3D3D3'
# cat_colors['PBMC'] = '#FF0000'
# cat_colors['Plaque'] = '#2a3439'

In [19]:
x_dim = 2000
y_dim = 2000

fig = plt.figure(animation_duration=2000)

# will be re-used to reset cell type colors
cell_type_colors = [cat_colors[df_meta.loc[x, 'Cell Type']] for x in df_meta.index.tolist()]
tissue_colors = [cat_colors[df_meta.loc[x, 'Tissue']] for x in df_meta.index.tolist()]

def_tt = bqplot.Tooltip(fields=['name'], formats=[''])

scatter = plt.scatter(df_meta['merge-umap-x-pos'], 
                      df_meta['merge-umap-y-pos'],
                      display_names=False, 
                      default_size = 35,
#                       size=size_values.values.tolist(),                                            
                      tooltip=def_tt,
                      names=df_meta.index.tolist(),
                      colors=cell_type_colors)



top_margin = 200
inst_width = 800
inst_height = 1000
fig.layout.min_height = str(inst_height) + 'px'
fig.layout.min_width  = str(inst_width) + 'px'

plt.xlim(0, 2.0*x_dim)
plt.ylim(0, 2.0*y_dim)
fig.title = 'UMAP'
fig.fig_margin = {'top': top_margin, 'bottom': 5, 'left': 5, 'right': 5}

In [20]:
# scatter.on_hover(callback=scatter_observe)
scatter.on_element_click(callback=scatter_observe)

In [21]:
default_opacity = 1

In [22]:
def cat_highlight(inst_value):
    
  
    
    cols = df_data.columns.tolist()

    if inst_value == 'reset_cats':

        scatter.colors = cell_type_colors
        list_opacities = [default_opacity for x in cols]
        scatter.default_opacities = list_opacities
        
    else:
        
        inst_cat_title = inst_value.split(': ')[0]
        
        if inst_cat_title == 'Cell Type':
            scatter.colors = cell_type_colors
        elif inst_cat_title == 'Tissue' : 
            scatter.colors = tissue_colors
        
        inst_cat = inst_value.split(': ')[1]

        list_opacities = []

        for inst_label in cols:
            inst_opacity = 0.15
            if df_meta.loc[inst_label, inst_cat_title] == inst_cat:
                inst_opacity = 1
            list_opacities.append(inst_opacity)        

         
    scatter.default_opacities = list_opacities

In [23]:
def on_value_change(change):
    # print(change['new'])
    
    col_nodes = net.viz['col_nodes']
    col_names = [x['name'] for x in col_nodes]    
    
    if change['new'] == 'null':
        
        # category highlight
        cat_highlight('reset_cats')
        fig.title = 'UMAP'
        
    else: 
        
        # click category
        if 'cat-' in change['new']:
            
            
            if ' -> ' in change['new']:
                inst_cat = change['new'].split(' -> ')[1]

                # print('on_value_change', inst_cat)
                cat_highlight(inst_cat)       

                fig.title = 'UMAP: ' + str(inst_cat.split(': ')[1])
            
        # mousing over marker
        elif 'row-label' in change['new']:
            
            inst_marker = change['new'].split(' -> ')[1]
            ser_opacity = df_data.loc[inst_marker]
            ser_opacity = ser_opacity / float(np.abs(ser_opacity.max()))
            
            fig.title = 'UMAP: ' + inst_marker
            
            # loop up opacities
            list_opacities = []
            rows = ser_opacity.index.tolist()
            
            list_marker_colors = []
            
            for inst_name in df_data.columns.tolist():
                
                inst_opacity = ser_opacity[inst_name]
                
                list_opacities.append(np.abs(inst_opacity))
                
                if inst_opacity >= 0:
                    list_marker_colors.append('red')
                else:
                    list_marker_colors.append('blue')
                    
            scatter.default_opacities = list_opacities
            scatter.colors = list_marker_colors
            
        elif 'col-dendro' in change['new']:
            # print('found col dendro!!!!!!!!!!!!!!!!!!!!')
            
            found_indexes = [int(x) for x in change['new'].split(' -> ')[1].split(',')]
            
            # the columns are not in the same order necessarily
            found_barcodes = []
            for inst_index in found_indexes:
                found_barcodes.append(col_names[inst_index])
                            
            # set to default cell type colors
            scatter.colors = cell_type_colors
            
            list_opacities = [1.0 if x in found_barcodes else 0.1 for x in df_meta.index.tolist()]
            scatter.default_opacities = list_opacities
            
            fig.title = 'UMAP: Dendrogram Cluster'
            
        else:
            # print('reset color and opacity')
            scatter.colors = cell_type_colors
            scatter.default_opacities = [1.0] * df_meta.shape[0]

In [24]:
net.viz = net.load_json_to_dict('../data/citeseq-dashboard/merge_viz.json')
net.widget()
net.widget_instance.observe(on_value_change, names='value')

In [25]:
from IPython.display import display
umap_merge_button = widgets.Button(description="UMAP: Merge")
umap_adt_button = widgets.Button(description="UMAP: ADT")
umap_gex_button = widgets.Button(description="UMAP: GEX")

# display(button)

def on_merge_button_click(b):
    #print("ADT")
    scatter.x = df_meta['merge-umap-x-pos']            
    scatter.y = df_meta['merge-umap-y-pos']

def on_adt_button_click(b):
    #print("ADT")
    scatter.x = df_meta['adt-umap-x-pos']            
    scatter.y = df_meta['adt-umap-y-pos']

    
def on_gex_button_click(b):
    #print("GEX")
    scatter.x = df_meta['gex-umap-x-pos'] 
    scatter.y = df_meta['gex-umap-y-pos']

    

umap_merge_button.on_click(on_merge_button_click)    
umap_adt_button.on_click(on_adt_button_click)
umap_gex_button.on_click(on_gex_button_click)

# HBox([location_button, umap_button])

In [26]:
button_section = widgets.HBox([umap_merge_button, umap_adt_button, umap_gex_button])
left_box = widgets.VBox([button_section, fig])
right_box = widgets.VBox([net.widget_instance])

# CITE-seq Atherosclerotic Plaques and PBMC

In [27]:
widgets.HBox([left_box, right_box])